<a href="https://colab.research.google.com/github/Matrixmax/Dive_into_DeepLearning/blob/main/0x20_GPU%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

到目前为止，我们一直在使用CPU计算。对复杂的神经网络和大规模的数据来说，使用CPU来计算可能不够高效。在本节中，我们将介绍如何使用单块NVIDIA GPU来计算。所以需要确保已经安装好了PyTorch GPU版本。准备工作都完成后，下面就可以通过nvidia-smi命令来查看显卡信息了。

In [2]:
!nvidia-smi

Sun Feb 14 09:06:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. 计算设备
PyTorch可以指定用来存储和计算的设备，如使用内存的CPU或者使用显存的GPU。默认情况下，PyTorch会将数据创建在内存，然后利用CPU来计算。

使用 torch.cuda.is_available() 查看GPU是否可用


In [3]:
import torch
from torch import nn

torch.cuda.is_available()


True

In [4]:
# 查看GPU数量
torch.cuda.device_count()

1

In [5]:
# 查看当前GPU索引号，索引从0开始
torch.cuda.current_device()

0

In [6]:
# 根据索引号查看GPU名字
torch.cuda.get_device_name(0)

'Tesla T4'

## 2.Tensor的GPU计算
默认情况下，Tensor会被存在内存上，因此，之前我们每次打印tensor的时候看不到GPU相关标识

In [7]:
x = torch.tensor([1,2,3])
x

tensor([1, 2, 3])

使用.cuda()可以将CPU上的Tensor转换（复制）到GPU上，如果有多块GPU，我们用.cuda(i)来表示第$i$块GPU以及相关的显存（$i$从0开始）且cuda(0)和cuda()等价

In [8]:
x = x.cuda(0)
x

tensor([1, 2, 3], device='cuda:0')

In [9]:
# 我们可以通过Tensor的device属性来查看该Tensor所在的设备。
x.device

device(type='cuda', index=0)

In [12]:
# 我们可以直接在创建的时候就指定设备。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.tensor([1,2,3],device = device)

# or
# x = torch.tensor([1,2,3]).to(device)
x

tensor([1, 2, 3], device='cuda:0')

In [13]:
# 如果对在GPU上的数据进行运算，那么结果还是存放在GPU上。
y  = x**2
y

tensor([1, 4, 9], device='cuda:0')

**需要注意的是，存储在不同位置中的数据是不可以直接进行计算的。即存放在CPU上的数据不可以直接与存放在GPU上的数据进行运算，位于不同GPU上的数据也是不能直接进行计算的。**

In [14]:
z = y+x.cpu()

RuntimeError: ignored

## 3.模型的GPU计算
同Tensor类似，PyTorch模型也可以通过.cuda转换到GPU上。我们可以通过检查模型的参数的device属性来查看存放模型的设备。

In [15]:
net = nn.Linear(3,1)
list(net.parameters())[0].device

device(type='cpu')

In [16]:
# 可见模型在CPU上，将其转换到GPU上:
net.cuda()
list(net.parameters())[0].device

device(type='cuda', index=0)

**同样的，我么需要保证模型输入的Tensor和模型都在同一设备上，否则会报错。**

In [17]:
x = torch.rand(2,3).cuda()
net(x)

tensor([[0.4562],
        [0.6498]], device='cuda:0', grad_fn=<AddmmBackward>)